# **Cloning Repositories and installing Packages**

In [2]:
!git clone "https://github.com/CorentinJ/Real-Time-Voice-Cloning.git"

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 2665, done.
remote: Total 2665 (delta 0), reused 0 (delta 0), pack-reused 2665
Receiving objects: 100% (2665/2665), 360.96 MiB | 24.07 MiB/s, done.
Resolving deltas: 100% (1482/1482), done.


In [14]:
!pip install pydub

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q -r /content/Real-Time-Voice-Cloning/requirements.txt

     |████████████████████████████████| 11.2 MB 17.6 MB/s 
     |████████████████████████████████| 15.3 MB 41.0 MB/s 
     |████████████████████████████████| 3.1 MB 50.6 MB/s 
     |████████████████████████████████| 8.3 MB 38.5 MB/s 
     |████████████████████████████████| 38.1 MB 8.7 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 235 kB 47.0 MB/s 
     |████████████████████████████████| 138 kB 49.5 MB/s 
     |████████████████████████████████| 676 kB 39.3 MB/s 
     |████████████████████████████████| 66 kB 4.4 MB/s 
     |████████████████████████████████| 899 kB 31.2 MB/s 
     |████████████████████████████████| 59.9 MB 1.3 MB/s 
     |████████████████████████████████| 338 kB 48.1 MB/s 
     |████████████████████████████████| 1.1 MB 40.2 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
ERROR: pip's dependenc

In [4]:
!apt-get install -qq libportaudio2

Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



# Create Model Folder and download Pretrained Models (models are in the drive)

Original Models can be downloaded from https://github.com/CorentinJ/Real-Time-Voice-Cloning/wiki/Pretrained-models

In [6]:
!mkdir Models
%cd /content/Models/

/content/Models


In [9]:
import shutil 
shutil.copy('/content/drive/MyDrive/pretrained/synthesizer.pt','/content/Models')
shutil.copy('/content/drive/MyDrive/pretrained/vocoder.pt','/content/Models')


'/content/Models/vocoder.pt'

In [11]:
%cd /content/Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


# **Loading the models**

In [13]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("/content/Models/encoder.pt")
vocoder_weights = Path("/content/Models/vocoder.pt")
syn_dir = Path("/content/Models/synthesizer.pt")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
# synthesizer.model_fpath
synthesizer.load()
vocoder.load_model(vocoder_weights)

Loaded encoder "encoder.pt" trained to step 1567401
Synthesizer using device: cuda
Trainable Parameters: 30.870M
Loaded synthesizer "synthesizer.pt" trained to step 295000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at /content/Models/vocoder.pt


Enter The text That needs to be cloned also enter the voice sample in wav that needs to be cloned

In [15]:
text = '''
This model was trained on indian english

'''

##upload the sample voice file and paste the path here
in_fpath = Path("/content/26e08c23-ccba-435d-9a53-d8bdcc43ff87-byVC.wav")
out_fpath = '/content/speech.wav'


**(Optional for the changing mp3 files into wav)**

In [ ]:
#converting mp3 to wav

from pydub import AudioSegment
  
# convert mp3 to wav file
sound = AudioSegment.from_mp3(in_fpath)
sound.export(out_fpath, format="wav")
speech = sound
speech.export(out_fpath)
in_fpath = out_fpath

# **Final Cloned voice**

In [16]:
reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)
with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
generated_wav = vocoder.infer_waveform(specs[0])
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
display(Audio(generated_wav, rate=synthesizer.sample_rate))

{| ████████████████ 47500/48000 | Batch Size: 5 | Gen Rate: 3.3kHz | }